In [32]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


rng = 69

In [33]:
#df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")
df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")

In [34]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away','win_or_lose_Home','goalsTotal_Away'])

In [35]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [36]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [37]:
corr_matrix['goalsTotal_Home'].sort_values(ascending=False)

goalsTotal_Home              1.000000
goalsAgainstAvg_Home         0.052154
goalsForAvg_Away             0.048619
goalsAgainstAvg_Away         0.040320
reboundsForAvg_Away          0.020040
reboundsForAvg_Home          0.016664
penaltiesForTotal            0.011857
fenwickPercentageAvg_Away    0.003288
penaltiesAgainstTotal        0.001298
pointsFromGameAvg_Away      -0.002941
goalsForAvg_Home            -0.003039
seasonPointsPerGame_Away    -0.006146
hitsDiffAvg_Away            -0.007814
fenwickPercentageAvg_Home   -0.008797
seasonPointsPerGame_Home    -0.022595
pointsFromGameAvg_Home      -0.024717
hitsDiffAvg_Home            -0.029000
Name: goalsTotal_Home, dtype: float64

In [38]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [39]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [40]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('goalsTotal_Home', axis=1)
nhl_labels = train_set['goalsTotal_Home'].copy()

X_train = train_set.drop('goalsTotal_Home', axis=1)
Y_train = train_set['goalsTotal_Home'].copy()

X_valid = train_set.drop('goalsTotal_Home', axis=1)
Y_valid = train_set['goalsTotal_Home'].copy()

X_test = test_set.drop('goalsTotal_Home', axis=1)
Y_test = test_set['goalsTotal_Home'].copy()

df_Final = df_Final.drop('goalsTotal_Home', axis=1)

In [41]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [42]:
# set epochs to use same num in all calls
n_epochs = 50

In [43]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3517 entries, 2920 to 4041
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   seasonPointsPerGame_Home   3517 non-null   float64
 1   goalsForAvg_Home           3517 non-null   float64
 2   goalsAgainstAvg_Home       3517 non-null   float64
 3   hitsDiffAvg_Home           3517 non-null   float64
 4   pointsFromGameAvg_Home     3517 non-null   float64
 5   fenwickPercentageAvg_Home  3517 non-null   float64
 6   reboundsForAvg_Home        3517 non-null   float64
 7   seasonPointsPerGame_Away   3517 non-null   float64
 8   goalsForAvg_Away           3517 non-null   float64
 9   goalsAgainstAvg_Away       3517 non-null   float64
 10  hitsDiffAvg_Away           3517 non-null   float64
 11  pointsFromGameAvg_Away     3517 non-null   float64
 12  fenwickPercentageAvg_Away  3517 non-null   float64
 13  reboundsForAvg_Away        3517 non-null   float64

In [44]:
input_shape=[X_train.shape[1]]

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=input_shape),
    tf.keras.layers.Dense(15, activation='selu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(15, activation='selu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='relu')  # Output layer for regression
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [45]:
def build_model(n_neurons=50, n_hidden=3, dropout_rate=0.4, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu'))
        model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    return model

In [46]:
best_model = build_model(dropout_rate=0.4, learning_rate=0.001157866133544249, n_hidden=2, n_neurons=25)

In [47]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=best_model)


In [48]:

from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)
# Using a Callback to run all epochs and only save the best one

early_stopping = EarlyStopping(monitor='val_loss', patience=15)
save_file = r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\Best_Over_Under_Model.keras"
checkpoint_cb = keras.callbacks.ModelCheckpoint(save_file, save_best_only = True)


keras_reg.fit(X_train_prepared, Y_train, 
              validation_data = (X_valid_prepared, Y_valid),
              callbacks=[early_stopping,checkpoint_cb],epochs=n_epochs)

Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 41.4942 - mae: 5.7417 - val_loss: 23.2211 - val_mae: 4.1956
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22.2569 - mae: 3.9270 - val_loss: 7.4111 - val_mae: 2.1351
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.6114 - mae: 2.8177 - val_loss: 5.8333 - val_mae: 1.9067
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.1264 - mae: 2.5234 - val_loss: 5.6610 - val_mae: 1.8829
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7311 - mae: 2.4945 - val_loss: 5.5975 - val_mae: 1.8765
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.7071 - mae: 2.3402 - val_loss: 5.5454 - val_mae: 1.8727
Epoch 7/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.1263 - mae: 2.2551 - val_loss: 5.4876 - val_mae: 1.8666
Epoch 8/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.0034 - mae: 2.2404 - val_loss: 5.4820 - val_mae: 1.8667
Epoch 9/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

KerasRegressor(
	model=<Sequential name=sequential_2, built=True>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [49]:
keras_reg.model_.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 25)                  │             425 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 25)                  │             650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305 (12.91 KB)

 Trainable params: 1,101 (4.30 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,204 (8.61 KB)

In [50]:
from sklearn.model_selection import cross_val_score

# Assuming X and y are your feature matrix and target vector
scores = cross_val_score(keras_reg, X_train_prepared, Y_train, cv=5, scoring='neg_mean_squared_error')

# Convert scores to positive values
mse_scores = -scores
print(f'MSE Scores: {mse_scores}')
print(f'Mean MSE: {np.mean(mse_scores)}')

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - loss: 6.0805 - mae: 1.9599
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.4213 - mae: 2.0264  
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - loss: 5.6988 - mae: 1.8904
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 662us/step - loss: 5.8277 - mae: 1.9272
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5.8726 - mae: 1.9234 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
MSE Scores: [5.90088269 4.91947239 5.221654   5.46472479 5.76884546]
Mean MSE: 5.455115866065839


In [51]:
X_train_pred = keras_reg.predict(X_train_prepared)
X_test_pred = keras_reg.predict(X_test_prepared)

# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, X_train_pred)
mae = mean_absolute_error(Y_train, X_train_pred)

print(f'Train MSE: {mse}') # Sensitive to outliers
print(f'Train MAE: {mae}')

mse = mean_squared_error(Y_test, X_test_pred)
mae = mean_absolute_error(Y_test, X_test_pred)

print(f'Test MSE: {mse}') # Sensitive to outliers
print(f'Test MAE: {mae}')

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step
Train MSE: 5.447457203351581
Train MAE: 1.860606435553947
Test MSE: 5.196016353663203
Test MAE: 1.854864213141528


In [52]:
# Try usinga simple random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Initialize and train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_prepared, Y_train)

# Make predictions
Y_pred = rf_model.predict(X_test_prepared)

# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 5.282719659090908


In [53]:
#df_Predict = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")

In [54]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_reg.predict(X_real_predict))).T

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


array([['COL', 6.377510070800781],
       ['PIT', 7.164556503295898],
       ['TOR', 6.882906913757324]], dtype=object)